In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def get_consent_task():
    return {
        "name": "Consent",
        "type": "InstructionsTask",
        "prerequisite": True,
        "content": {
            "type": "link",
            "url": "$$www$$/consent.md"
        },
        "form": {
            "fields": [
                {
                    "name": "consent1",
                    "label": "I have read and understood the study information dated [DD/MM/YYYY], or it has been read to me. I have been able to ask questions about the study and my questions have been answered to my satisfaction.",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent2",
                    "label": "I consent voluntarily to be a participant in this study and understand that I can refuse to answer questions and I can withdraw from the study at any time, without having to give a reason. ",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent3",
                    "label": "I understand that taking part in the study involves providing data ratings or annotations which will be recorded but not linked to my identity, along with data about my interaction with the web interface.",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent4",
                    "label": "I understand that information I provide will be used for scientific publications.",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent5",
                    "label": "I understand that personal information collected about me that can identify me, such as [e.g. my name or where I live], will not be asked nor stored.",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                },
                {
                    "name": "consent6",
                    "label": "I give permission for the annotations that I provide to be archived in networked storage at Delft University of Technology in anonymized form so it can be used for future research. I understand that data will not be made public but may be shared with researchers under an End User License Agreement (EULA).",
                    "required": True,
                    "input": {
                        "inputType": "Checkbox.Group",
                        "options": [
                            {
                                "label": "Yes",
                                "value": "yes"
                            }
                        ]
                    }
                }
            ]
        }
}

In [3]:
def get_instructions_task(file, type='video'):
    titles = {
        'video': 'Video-only instructions',
        'audio': 'Audio-only instructions',
        'av': 'Audiovisual instructions'
    }
    return {
        "name": titles[type],
        "type": "InstructionsTask",
        "prerequisite": False,
        "content": {
            "type": "link",
            "url": f"$$www$$/{file}"
        }
    }

In [12]:
def get_rating_tasks(examples, fpath='laughter_examples/av', ini=1):
    tasks = []
    for i, ex in enumerate(examples):
        # add the continous task
        tasks.append({
            "type": "Continuous1DTask",
            "name": f"#{ini+i}: Recognition",
            "media": {
                "type": "video",
                "url": f"$$www$$/{fpath}/{ex.get_file_id()}.mp4",
                "fps": 25
            },
            "intensityInput": {
                "mode": "binary"
            },
            "instructions": "Find the person indicated with a red rectangle. Play the video and press indicate the ocurrence of laughter by pressing \"q\"."
        })
        # add the rating task
        tasks.append({
            "type": "QuestionnaireTask",
            "name": f"#{ini+i}: Rating",
            "form": {
            "fields": [
                {
                  "name": "intensity",
                  "label": "Laughter intensity: ",
                  "required": True,
                  "input": {
                    "inputType": "Slider",
                    "defaultValue": 50
                  }
                },
                {
                  "name": "confidence",
                  "label": "Annotation confidence: ",
                  "required": True,
                  "input": {
                    "inputType": "Slider",
                    "defaultValue": 50
                  }
                }
            ]
            },
            "media": {
                "type": "video",
                "url": f"$$www$$/{fpath}/{ex.get_file_id()}.mp4",
            },
            "instructions": "Find the person indicated with a red rectangle. Play the video and pay attention to this person. Give the required ratings on the right."
        })
    return tasks

In [5]:
import json
import pandas as pd
sys.path.append('../lared')
from dataset.example import VideoExample, AudioExample, FullExample
cloud_data_path = '/home/jose/drive/data/lared_laughter'
examples_df = pd.read_csv(os.path.join(cloud_data_path, 'pilot/laughter_examples/examples.csv'), index_col=0)

In [6]:
filtered_rows = examples_df[examples_df['rect'].notnull()]
examples = [FullExample(**ex[1].to_dict()) for ex in filtered_rows.iterrows()]

In [7]:
examples_df.head()

,id,pid,cam,valid,ini_time,end_time,labels,rect,ini,len,hash
0,NaN,1,2,True,1026.363079,1032.543362,"{'vad': None, 'label': None, 'vad_seg': None}",NaN,NaN,NaN,6204e39b1fba6a5f5b3472e5c9ed0e6e932025d8be03e2...
1,NaN,1,3,True,1026.363079,1032.543362,"{'vad': None, 'label': None, 'vad_seg': None}",NaN,NaN,NaN,1ee2b341f7d4d829f53bcdb54971b2b71cd974869ed603...
2,NaN,1,4,True,1026.363079,1032.543362,"{'vad': None, 'label': None, 'vad_seg': None}",NaN,NaN,NaN,261654fdb4b038241b0777e69120a7ff33e36200bbd20c...
3,NaN,1,2,True,1047.638924,1054.518364,"{'vad': None, 'label': None, 'vad_seg': None}",NaN,NaN,NaN,ceff382e64c66da3c04cdbf680d99dbcc5a17bc4cb1855...
4,NaN,1,3,True,1047.638924,1054.518364,"{'vad': None, 'label': None, 'vad_seg': None}",NaN,NaN,NaN,90a90cef2c2cde2ccd167797d0e538d6ec89e8005e68ca...


In [8]:
import random

In [9]:
random.shuffle(examples)

In [10]:
examples[0].hash

'63acb6d5db8d9da978323d606533e178281d9626cba678fa226fb94fe01820b1'

In [18]:
project = {
    "name": "Pilot",
    "id": "pilot",
    "email": "example@example.com",
    "hits": [
        {
            "id": "pilot",
            "name": "Pilot",
            "type": "annotation",
            "interface": {
                "showProgress": True
            },
            "tasks": [
                get_consent_task(),

                get_instructions_task('video_instructions.md', 'video'),
                *get_rating_tasks(examples[0:46], fpath='laughter_examples/video', ini=1),

                get_instructions_task('audio_instructions.md', 'audio'),
                *get_rating_tasks(examples[46:46+45], fpath='laughter_examples/aiv', ini=47),

                get_instructions_task('av_instructions.md', 'av'),
                *get_rating_tasks(examples[46+45:], fpath='laughter_examples/av', ini=92)
            ]
        },
        
    ]
}

In [19]:
json.dump(project, open('pilot.covfee.json', 'w'), indent=2)